In [10]:
import os
import numpy as np
from PIL import Image

def load_data_with_paths(data_dir="dataset", img_size=(64, 64)):
    sets = ["train", "test"]
    classes = ["no_face", "face"]

    data = {}
    paths_data = {}

    for s in sets:
        X = []
        y = []
        paths = []

        for label, cls in enumerate(classes):
            class_dir = os.path.join(data_dir, s, cls)
            if not os.path.exists(class_dir):
                continue

            for file in os.listdir(class_dir):
                img_path = os.path.join(class_dir, file)
                try:
                    img = Image.open(img_path).convert("RGB")
                    img = img.resize(img_size)
                    X.append(np.array(img))
                    y.append(label)
                    paths.append(img_path) 
                except:
                    continue  # skip corrupted images

        data[s] = (np.array(X), np.array(y))
        paths_data[s] = np.array(paths)

    train_x_orig, train_y_orig = data["train"]
    test_x_orig, test_y_orig = data["test"]

    train_paths = paths_data["train"]
    test_paths = paths_data["test"]

    # Normalize
    train_x = train_x_orig.astype(np.float32) / 255.0
    test_x  = test_x_orig.astype(np.float32) / 255.0

    # Labels
    train_y = train_y_orig.T
    test_y  = test_y_orig.T

    return train_x, train_y, train_paths, test_x, test_y, test_paths, np.array(classes)


In [11]:
train_x, train_y, train_paths, test_x, test_y, test_paths, classes = load_data_with_paths()
print("Number of test paths:", len(test_paths))
print("First test path:", test_paths[0])

Number of test paths: 287
First test path: dataset\test\no_face\010_0006.jpg


In [48]:
known_faces = {}

In [59]:
import face_recognition
import numpy as np
import matplotlib.pyplot as plt
import os

def test_face(img_path):
    img = face_recognition.load_image_file(img_path)
    encodings = face_recognition.face_encodings(img)
    
    if len(encodings) == 0:
        print("No face detected")
        return
    
    new_encoding = encodings[0]
    
    # If no known faces, ask user to add
    if len(known_faces) == 0:
        add_name = input("No known faces! Enter name to add this face to the database: ")
        if add_name != "":
            known_faces[add_name] = new_encoding
            print(f"Added {add_name} to known faces.")
            return
        print("No new face added ")

    # Compare with known faces
    for name, known_encoding in known_faces.items():
        distance = np.linalg.norm(new_encoding - known_encoding)
        print(f"Distance to {name}: {distance:.3f}")
        if distance < 0.3:
            print(f"Match found: {name}")
            return
        print("No match found")

   


In [61]:
import random

idx = random.randint(0, len(test_paths)-1)
img_path = test_paths[idx]
test_face("C:/Users\gatsi\Downloads\tiakola.jpg")
print(img_path)
print(known_faces)


No known faces! Enter name to add this face to the database:  


No new face added 
dataset\test\face\202428.jpg
{}


In [55]:
known_faces.clear()
print(known_faces)

{}
